Este modelo de Machine Learning, busca realizar un sistema de recomendacion para usuarios que estan empezando o usuarios que ya han visto varios anime y no saben cual será el siguiente, metiendonos en el papel de Data Scientist de un estudio de animación nos piden realizar este modelo para recobrar información sobre los gustos de los usuarios mientras se le ofrece un servivio al usuario

## Librery

In [1]:
import sys 
import os  # obtener el path del jupyter tenemos esto 
root_path = os.path.dirname(os.getcwd())  
root_path  
sys.path.append(root_path)  
sys.path
print('.')

.


In [2]:
import os # this makes better error tracing 
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
from Utils.Librerias import *
from Utils.Path import *
from Utils.Visualizacion import *
from Utils.Limpieza import *
init_notebook_mode(connected=True)
cf.go_offline()
%matplotlib inline

C:\Users\jamr1\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jamr1\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\jamr1\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


## Data Load 

In [4]:
DF_AnimeList = pd.read_csv(AnimeList)
DF_Rating = pd.read_csv(Rating)

## Data Clean 

Modificamos la fecah para que solo sea el año con una función creada llamada aired_to_year y only_year

In [5]:
aired_to_year(DF_AnimeList,'aired_string')
MAL = only_year(DF_AnimeList,'Year_Aired')

Eliminamos las columnas aired_string y Year_Aired, reemplazamos en la columna
 source los elementos 4-koma manga, web manga, digital manga, por manga, 
 eliminamos de la columna type el elemento music, quitamos los nulos de la columna
 genre, de la columna year eliminamos los unknown y creamos columnas por cada
 elemento de genero donde 1 es que aplica y 0 que no


In [6]:
MAL = Clean_MAL(MAL)

Ahora vamos a limpiar los datos de usuario para ello llenaremos la columna
my_rewatching con 0, eliminamos los usuarios que tengan username con null,
y hacemos un reset_index

In [7]:
MAL_User = Clean_MAL_User(DF_UserAnimeList)

Se cambian por -1 los valores con NAN para no eliminar datos

In [8]:
MAL_Rating = Clean_MAL_Rating(DF_Rating)

Limitamos el dataframe para hacer las pruebas con mas facilidad

In [9]:
# MAL = MAL[MAL['type'] == 'TV']

## Merging Data

In [10]:
MAL_Full = MAL_Rating.merge(MAL, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
MAL_Full = MAL_Full[['user_id', 'title', 'score']]

Limitamos el dataframe con solo los primeros 5000 usuarios para pruebas

In [11]:
# MAL_Full= MAL_Full[MAL_Full.user_id <= 7500]

## Anime System Recommendation

Lo primero es realizar una pivot table para similitudes

In [12]:
MAL_Pivot = MAL_Full.pivot_table(index=['user_id'], columns=['title'], values='score')

Los siguientes pasos serian:
- Normalizar
- Tratar los NAN

In [13]:
MAL_Pivot_E = MAL_Pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

In [14]:
MAL_Pivot_E.fillna(0, inplace=True)

Realizamos una transpuesta para que sea mas facil de ver

In [15]:
MAL_Pivot_E = MAL_Pivot_E.T

In [16]:
MAL_Pivot_Sparse = sp.sparse.csr_matrix(MAL_Pivot_E.values)

In [17]:
MAL_Pivot_Sparse

<7079x73487 sparse matrix of type '<class 'numpy.float64'>'
	with 7518295 stored elements in Compressed Sparse Row format>

Cosine Similarity Model

In [18]:
SYCREC = cosine_similarity(MAL_Pivot_Sparse)

In [19]:
SYCREC_DF = pd.DataFrame(SYCREC, index = MAL_Pivot_E.index, columns = MAL_Pivot_E.index)

In [23]:
SYCREC_DF

title,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi","""Bungaku Shoujo"" Memoire","""Bungaku Shoujo"" Movie","""Eiji""",.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,.hack//Quantum: Sore ike! Bokura no Chimuchimu-chan!!,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,.hack//Tasogare no Udewa Densetsu: Offline de Aimashou,.hack//Unison,0-sen Hayato,0-sen Hayato Pilot,009-1,009-1: R&B,07-Ghost,1+2=Paradise,100-man-nen Chikyuu no Tabi: Bander Book,1000-nen Joou: Queen Millennia,11-nin Iru!,11eyes,11eyes Picture Drama,11eyes: Momoiro Genmutan,12-sai.,12-sai. 2nd Season,12-sai.: Chicchana Mune no Tokimeki,12-sai.: Chicchana Mune no Tokimeki 2nd Season,15 Bishoujo Hyouryuuki,21-ji no Onna: Newscaster Katsuragi Miki,3-tsu no Hanashi,3-tsu no Kumo,30-sai no Hoken Taiiku,3Ping Lovers!☆Ippu Nisai no Sekai e Youkoso♪ The Animation,3x3 Eyes,3x3 Eyes Seima Densetsu,4.Eyes,8-gatsu no Symphony: Shibuya 2002-2003,91 Days,91 Days Recap,A Kite,A-Channel,A-Channel+smile,A-Channel: +A-Channel,A-Girl,A-Ko The Versus,A-Size Classmate,A.F,AD Police,AIKa,AKB0048,AKB0048: Next Stage,Aa Harimanada,Aa! Megami-sama!,Aa! Megami-sama! (2011),Aa! Megami-sama! (TV),Aa! Megami-sama! (TV) Specials,Aa! Megami-sama! Movie,Aa! Megami-sama!: Chichaitte Koto wa Benri da ne,Aa! Megami-sama!: Sorezore no Tsubasa,Aa! Megami-sama!: Sorezore no Tsubasa Specials,Aa! Megami-sama!: Tatakau Tsubasa,Abarenbou Rikishi!! Matsutarou,Abashiri Ikka,Abenobashi Mahou☆Shoutengai,Absolute Duo,Abunai Sisters: Koko & Mika,Accel World,Accel World EX,Accel World: Acchel World.,Accel World: Infinite∞Burst,Accelerando: Datenshi-tachi no Sasayaki,Acchi Kocchi,Acchi Kocchi (TV),Acchi Kocchi (TV): Place=Princess,Ace wo Nerae!,Ace wo Nerae! 2,Active Raid: Kidou Kyoushuushitsu Dai Hachi Gakari,Active Raid: Kidou Kyoushuushitsu Dai Hachi Gakari 2nd,Ad Lib Anime Kenkyuujo,Adachi-ga Hara,Adesugata Mahou no Sannin Musume,Adventures in Beauty Wonderland,Afro Samurai,Afro Samurai Movie,Afro Samurai Pilot,Afro Samurai: Resurrection,After... The Animation,Agatha Christie no Meitantei Poirot to Marple,Ahiru Rikusentai,Ahiru no Otegara,Ai,Ai (ONA),Ai City,Ai Doll,Ai Mai! Moe Can Change!,Ai Shimai 2: Futari no Kajitsu,Ai Shimai Tsubomi... Kegashite Kudasai,Ai Shimai: Futari no Kajitsu,Ai Shoujo Pollyanna Story,Ai Sky Tree 21,Ai Tenchi Muyou!,Ai Tenshi Densetsu Wedding Peach,Ai Yori Aoshi,Ai Yori Aoshi: Enishi,Ai Yori Aoshi: Enishi - Miyuki,Ai Yori Aoshi: Yumegatari,Ai no Gakkou Cuore Monogatari,Ai no Katachi: Ecchi na Onnanoko wa Kirai... Desu ka?,Ai no Kusabi,Ai no Kusabi (2012),Ai no Senshi Rainbowman,Ai no Wakakusa Monogatari,Ai no Wakakusa Yama Monogatari,Ai to Yuuki no Pig Girl Tonde Buurin,Ai wa Kagi no Kazu dake... Fuuzoku Mansion,Aikatsu Stars! Movie,Aikatsu!,Aikatsu! Music Award: Minna de Shou wo MoracchaimaShow!,Aikatsu!: Nerawareta Mahou no Aikatsu! Card,Air,Air Gear,Air Gear: Kuro no Hane to Nemuri no Mori - Break on the Sky,Air Master,Air Movie,Air in Summer,Aisai Nikki,Aishen Qiaokeli-ing...,Aishite Knight,Aishiteruze Baby★★,Aitsu to Lullaby: Suiyobi no Cinderella,Aiura,Ajimu: Kaigan Monogatari,Ajin,Ajin Part 1: Shoudou,Ajin Part 2: Shoutotsu,Ajin Part 3: Shougeki,Akachan to Boku,Akagaki Genzou: Tokuri no Wakare,Akagami no Shirayuki-hime,Akagami no Shirayuki-hime 2nd Season,"Akagami no Shirayuki-hime: Nandemonai Takaramono, Kono Page",Akage no Anne,Akahori Gedou Hour Rabuge,Akai Hayate,Akai Kiba: Blue Sonnet,Akai Koudan Zillion,Akame ga Kill!,Akame ga Kill! Recap,Akame ga Kill! Theater,Akane Maniax,Akaneiro ni Somaru Saka,Akaneiro ni Somaru Saka: Hardcore,Akatsuki no Yona,Akazukin Chacha,Akazukin Chacha OVA,Akebi no Hana: Maho,Aki no Kanade,Aki no Puzzle,Aki-Sora,Aki-Sora: Yume no Naka,Akibakei Kanojo,Akihabara Dennou-gumi,Akihabara Dennou-gumi: 2011-nen no Natsuyasumi,Akikan!,Akikan!: Kan Ippatsu!? Onsen Panic,Akina to Onsen de H Shiyo!,Akira,Aku no Hana,Aku no Onna Kanbu,Aku no Onna Kanbu: 

In [20]:
def recomendador(ani_name):
    """
    Esta Función devuelve los 5 animes que mas se parezcan al introducido
    
    ejemplo:
    Entrada: 
    
    recomendador('Death Note')
    
    Salida: 
    
    Recomendado porque buscaste Death Note:

                    #1: Code Geass: Hangyaku no Lelouch, 55.49%
                    #2: Code Geass: Hangyaku no Lelouch R2, 52.29%
                    #3: Fullmetal Alchemist, 49.41%
                    #4: Shingeki no Kyojin, 49.16%
                    #5: Fullmetal Alchemist: Brotherhood, 47.62% 

               
    """
    
    number = 1
    print('Recomendado porque buscaste {}:\n'.format(ani_name))
    for anime in SYCREC_DF.sort_values(by = ani_name, ascending = False).index[1:6]:
        print(f'#{number}: {anime}, {round(SYCREC_DF[anime][ani_name]*100,2)}%')
        number +=1

In [24]:
recomendador('Dragon Ball')

Recomendado porque buscaste Dragon Ball:

#1: Dragon Ball Z, 75.71%
#2: Fullmetal Alchemist, 38.1%
#3: Death Note, 36.67%
#4: Digimon Adventure, 36.58%
#5: Dragon Ball Z Special 2: Zetsubou e no Hankou!! Nokosareta Chousenshi - Gohan to Trunks, 35.42%
